# PSO
- Initialization: A swarm of particles is initialized with random positions and velocities.
- Fitness Evaluation: Each particle’s fitness value is evaluated by passing it to the objective function.
- Update Personal Best: Each particle remembers the best position it has found so far (personal best).
- Update Global Best: The best position found by any particle in the swarm is remembered as the global best.
- Velocity Update: Each particle's velocity is updated based on its personal best position and the global best position.
- Position Update: Each particle's position is updated using its velocity.
- Termination: The algorithm stops when the maximum number of iterations is reached or an acceptable fitness value is found.

In [16]:
import random

# Distance matrix for a small graph (5 cities)
distance_matrix = [
    [0, 2, 9, 10, 7],
    [2, 0, 6, 4, 3],
    [9, 6, 0, 8, 5],
    [10, 4, 8, 0, 1],
    [7, 3, 5, 1, 0]
]

num_cities = len(distance_matrix)
num_particles = 30
num_iterations = 100
w = 0.5  # Inertia weight
c1 = 1.5 # Personal attraction weight
c2 = 1.5 # Global attraction weight

# Objective Function
def calculate_total_distance(tour):
    return sum(distance_matrix[tour[i]][tour[i + 1]] for i in range(len(tour) - 1)) + distance_matrix[tour[-1]][tour[0]]

# Initialize particles
particles = []
for _ in range(num_particles):
    tour = list(range(num_cities))
    random.shuffle(tour)
    particles.append({
        'position': tour,
        'velocity': [],
        'best_position': tour,
        'best_value': calculate_total_distance(tour)
    })

# Initialize global best
global_best_position = min(particles, key=lambda p: p['best_value'])['position']
global_best_value = calculate_total_distance(global_best_position)

# PSO Main loop
for iteration in range(num_iterations):
    for particle in particles:
        # Calculate fitness value
        current_value = calculate_total_distance(particle['position'])

        # Update personal best if the current solution is better
        if current_value < particle['best_value']:
            particle['best_value'] = current_value
            particle['best_position'] = particle['position'][:]

        # Update global best if the current solution is better
        if current_value < global_best_value:
            global_best_value = current_value
            global_best_position = particle['position'][:]

    # Update velocity and position for each particle
    for particle in particles:
        # Velocity in TSP is represented as a series of swaps
        velocity = []

        # Cognitive component: compare with personal best
        for i in range(num_cities):
            if particle['position'][i] != particle['best_position'][i]:
                swap_with = particle['best_position'].index(particle['position'][i])
                velocity.append((i, swap_with))

        # Social component: compare with global best
        for i in range(num_cities):
            if particle['position'][i] != global_best_position[i]:
                swap_with = global_best_position.index(particle['position'][i])
                velocity.append((i, swap_with))

        # Ensure the velocity is not larger than the actual list
        max_sample_size = min(len(velocity), int(w * len(velocity)) + random.randint(0, 1))

        # Apply a portion of velocity determined by weights w, c1, and c2
        if max_sample_size > 0:
            sampled_velocity = random.sample(velocity, max_sample_size)
        else:
            sampled_velocity = []

        # Update particle's position by applying swaps from velocity
        for (i, j) in sampled_velocity:
            particle['position'][i], particle['position'][j] = particle['position'][j], particle['position'][i]

        # Store the updated velocity
        particle['velocity'] = sampled_velocity

    # Display iteration info
    print(f"Iteration {iteration + 1}/{num_iterations}, Best Distance: {global_best_value}")

# Final best tour found
print(f"Global best tour found: {global_best_position} with distance: {global_best_value}")


Iteration 1/100, Best Distance: 21
Iteration 2/100, Best Distance: 21
Iteration 3/100, Best Distance: 21
Iteration 4/100, Best Distance: 21
Iteration 5/100, Best Distance: 21
Iteration 6/100, Best Distance: 21
Iteration 7/100, Best Distance: 21
Iteration 8/100, Best Distance: 21
Iteration 9/100, Best Distance: 21
Iteration 10/100, Best Distance: 21
Iteration 11/100, Best Distance: 21
Iteration 12/100, Best Distance: 21
Iteration 13/100, Best Distance: 21
Iteration 14/100, Best Distance: 21
Iteration 15/100, Best Distance: 21
Iteration 16/100, Best Distance: 21
Iteration 17/100, Best Distance: 21
Iteration 18/100, Best Distance: 21
Iteration 19/100, Best Distance: 21
Iteration 20/100, Best Distance: 21
Iteration 21/100, Best Distance: 21
Iteration 22/100, Best Distance: 21
Iteration 23/100, Best Distance: 21
Iteration 24/100, Best Distance: 21
Iteration 25/100, Best Distance: 21
Iteration 26/100, Best Distance: 21
Iteration 27/100, Best Distance: 21
Iteration 28/100, Best Distance: 21
I

# ACO
- Initialization: The problem is represented as a graph. The pheromone levels are initialized for each edge in the graph.
- Ant Solution Construction: Each ant starts at a random node and builds a solution based on pheromone strength and heuristic information.
- Pheromone Update: After all ants complete their tours, the pheromones are updated:
- Evaporation: Reduce the pheromone level over time to avoid convergence to a suboptimal solution.
- Reinforcement: Increase pheromone levels on paths that were part of good solutions.
- Iteration: Repeat the process until convergence or a maximum number of iterations is reached.

In [17]:
import random

# Distance matrix for a small graph (5 cities)
distance_matrix = [
    [0, 2, 9, 10, 7],
    [2, 0, 6, 4, 3],
    [9, 6, 0, 8, 5],
    [10, 4, 8, 0, 1],
    [7, 3, 5, 1, 0]
]

num_cities = len(distance_matrix)
num_ants = 10
num_iterations = 100
alpha = 1  # Influence of pheromone
beta = 5   # Influence of heuristic value (distance)
rho = 0.5  # Pheromone evaporation rate
pheromone_deposit = 100  # Amount of pheromone deposited by an ant

# Initialize pheromone matrix with a small positive value
pheromone_matrix = [[1 for _ in range(num_cities)] for _ in range(num_cities)]

def calculate_total_distance(tour):
    """ Calculate total distance for a given tour """
    return sum(distance_matrix[tour[i]][tour[i + 1]] for i in range(len(tour) - 1)) + distance_matrix[tour[-1]][tour[0]]

# Main ACO loop
for iteration in range(num_iterations):
    all_tours = []  # Store all tours of this iteration

    # Each ant constructs a solution
    for ant in range(num_ants):
        # Start from a random city
        start_city = random.randint(0, num_cities - 1)
        tour = [start_city]
        visited = set(tour)

        # Construct a complete tour
        for _ in range(num_cities - 1):
            current_city = tour[-1]
            # Calculate probabilities to move to each city
            probabilities = []
            for city in range(num_cities):
                if city not in visited:
                    pheromone = pheromone_matrix[current_city][city] ** alpha
                    heuristic = (1 / distance_matrix[current_city][city]) ** beta
                    probabilities.append(pheromone * heuristic)
                else:
                    probabilities.append(0)  # Already visited city has zero probability
            
            # Normalize probabilities
            total_probability = sum(probabilities)
            probabilities = [p / total_probability for p in probabilities]

            # Select the next city based on the calculated probabilities
            next_city = random.choices(range(num_cities), weights=probabilities, k=1)[0]
            tour.append(next_city)
            visited.add(next_city)

        all_tours.append(tour)

    # Update pheromones
    # Evaporation
    for i in range(num_cities):
        for j in range(num_cities):
            pheromone_matrix[i][j] *= (1 - rho)

    # Add pheromone based on the quality of the tours
    for tour in all_tours:
        tour_distance = calculate_total_distance(tour)
        pheromone_amount = pheromone_deposit / tour_distance
        for i in range(len(tour) - 1):
            pheromone_matrix[tour[i]][tour[i + 1]] += pheromone_amount
        pheromone_matrix[tour[-1]][tour[0]] += pheromone_amount

    # Find the best tour of the iteration
    best_tour = min(all_tours, key=calculate_total_distance)
    best_distance = calculate_total_distance(best_tour)

    # Print the iteration's best result
    print(f"Iteration {iteration + 1}/{num_iterations}, Best Distance: {best_distance}")

# Find the overall best tour
best_tour = min(all_tours, key=calculate_total_distance)
best_distance = calculate_total_distance(best_tour)

print(f"Best tour found: {best_tour} with distance: {best_distance}")


Iteration 1/100, Best Distance: 21
Iteration 2/100, Best Distance: 21
Iteration 3/100, Best Distance: 21
Iteration 4/100, Best Distance: 21
Iteration 5/100, Best Distance: 21
Iteration 6/100, Best Distance: 21
Iteration 7/100, Best Distance: 21
Iteration 8/100, Best Distance: 21
Iteration 9/100, Best Distance: 21
Iteration 10/100, Best Distance: 21
Iteration 11/100, Best Distance: 21
Iteration 12/100, Best Distance: 21
Iteration 13/100, Best Distance: 21
Iteration 14/100, Best Distance: 21
Iteration 15/100, Best Distance: 21
Iteration 16/100, Best Distance: 21
Iteration 17/100, Best Distance: 21
Iteration 18/100, Best Distance: 21
Iteration 19/100, Best Distance: 21
Iteration 20/100, Best Distance: 21
Iteration 21/100, Best Distance: 21
Iteration 22/100, Best Distance: 21
Iteration 23/100, Best Distance: 21
Iteration 24/100, Best Distance: 21
Iteration 25/100, Best Distance: 21
Iteration 26/100, Best Distance: 21
Iteration 27/100, Best Distance: 21
Iteration 28/100, Best Distance: 21
I